In [1]:
# -*- encoding: utf-8 -*-
import sys
import codecs
import csv
import MeCab
import os

In [2]:
reload(sys)
sys.setdefaultencoding('utf-8')

In [3]:
import re, pprint
def pp(obj):
    pp = pprint.PrettyPrinter(indent=4, width=160)
    str = pp.pformat(obj)
    return re.sub(r"\\u([0-9a-f]{4})", lambda x: unichr(int("0x"+x.group(1),16)), str)

In [4]:
# 读取停用词一览
def readStopWord(ssfile):
    ss = []
    fr = open(ssfile,"r")
    for line in fr.readlines():
        line = line.strip()
        if line != '':
            ss.append(line)
    fr.close()
    return ss

In [5]:
# 读取CSV文件
def readDataFile(fileName):
    with open(fileName,"r+") as csvfile:
        reader = csv.reader(csvfile)
        # skip header
        header = next(reader)   
#         print "Read article:",fileName
        # 读取内容
        for line in reader:
            return line

In [6]:
# 获取词汇本体
def get_surfaces(node):
    words = []
    while node:
#         word  = node.surface.decode('utf-8')
        word  = node.surface
        words.append(word)
        node = node.next
    return words

In [7]:
# 获取词性
def get_nouns(node):
    nouns = []
    while node:
#         noun = node.feature.split(",")[0].decode('utf-8')
        noun = node.feature.split(",")[0]
        nouns.append(noun)
        node = node.next
    return nouns

In [8]:
# 利用词性筛选词汇
def select_feature(data,attr):
    result = []
    pattern1 = re.compile(r'[^0-9]+')
    pattern2 = re.compile(ur'[^０１２３４５６７８９]+')
    pattern3 = re.compile(ur'[^\[\]!$%&\'\"\(\):\-\.,/;=<>]+')
    pattern4 = re.compile(ur'[^！　％、。）※「」（＞～』＜？－．♪【⇒∞★〇・⇔]+')
    for (w,a) in data:
        if a in attr:
#             print w,a
            # 过滤所有数字
            tmp = w.decode('utf-8')
            matcher1 = re.match(pattern1,w)
            matcher2 = re.match(pattern2,tmp)
            matcher3 = re.match(pattern3,tmp)
            matcher4 = re.match(pattern4,tmp)
            if (matcher1 is not None) and (matcher2 is not None) and (matcher3 is not None) and (matcher4 is not None):
#                 print w,a
                result.append((w,a))
    return result

In [9]:
# 把筛选以后的结果写入txt文件
def writeOutput(fileName,data,mode):
    fw = open(fileName,mode)
    line = ""
    for w in data:
        line = line + w + "\t"
    fw.writelines( line  + "\n")
    fw.close()

In [10]:
# 输出结果有两个文件
# 全量分词结果文件：ALLWORD_filename
# 词性筛选结果文件：mode_file (默认筛选名词 mode = "N")
# 筛选 形容詞 ：mode 指定 "A"
# 筛选 动词 ：mode 指定 "V"
# 例如：筛选 名词，形容词的场合 :mode = "NA"
def processFile(inPath,outPath,stopwords,mode="N"):
    
    # 获取文件列表
    fileList = os.listdir(inPath)
    
    for f in fileList:
        
        print "-----Processing File:",f
        # 编辑文件路径
        fileName = inPath + f
        # 获取文章类别(验证用)
        catlog = f.split(".")[0].split("-")[-1]
        # 获取文件内容
        content = readDataFile(fileName)
        # 获取文章标题
        title = catlog + "#" + content[0]
#         print "title=",title
        # 定义分词器
        mt = MeCab.Tagger('mecabrc')
        node = mt.parseToNode(content[1])
        words, nouns = get_surfaces(node), get_nouns(node)
        wordList = zip(words,nouns)

        attr = [u'名詞']
        
        if "A" in mode:
            # 筛选形容詞结果写入CSV
            attr.append(u'形容詞')
  
        if "V" in mode:
            # 筛选動詞结果写入CSV
            attr.append(u'動詞')
        
        # 筛选名词结果写入CSV
        selectList = select_feature(wordList,attr)

        #把文章标题加载每一行的第一个位置
        outWord = []
        outWord.append(title)
        # 去掉停用词
        for (w,a) in selectList:
#             print "w=",w
            if w not in stopwords:
                outWord.append(w)
        writeOutput(outPath,outWord,"a")

In [11]:
def processAll():
    # 读入停用词一览
    ssfile = "./conf/Japanese_stopwords.txt"
    ssList = readStopWord(ssfile)
    
    input_path = ["/home/hadoop/Crawler/bus_all/",
                  "/home/hadoop/Crawler/ent/",
                  "/home/hadoop/Crawler/ind/",
                  "/home/hadoop/Crawler/int/",
                  "/home/hadoop/Crawler/life/",
                  "/home/hadoop/Crawler/pol/",
                  "/home/hadoop/Crawler/prod/",
                  "/home/hadoop/Crawler/sci/",
                  "/home/hadoop/Crawler/soci/",
                  "/home/hadoop/Crawler/spo/"]  
    output_path = "/home/hadoop/Crawler/all.txt"
    
    # 筛选名词    
#     processFile(input_path,output_path,"N")
    # 筛选名词和形容次
#     processFile(input_path,output_path,"NA")
    # 筛选名词和形容次，动词
#     processFile(input_path,output_path,"NAV")
    for p in input_path:
        print "Process Folder:",p
        processFile(p,output_path,ssList,"NA")

In [12]:
def processTest():
    
    # 读入停用词一览
    ssfile = "./conf/Japanese_stopwords.txt"
    ssList = readStopWord(ssfile)
       
    input_path = ["/home/hadoop/DataSencise/text_data/test/"] 
    output_path = "/home/hadoop/DataSencise/text_data/test/test.txt"
    
    # 筛选名词    
#     processFile(input_path,output_path,"N")
    # 筛选名词和形容次
#     processFile(input_path,output_path,"NA")
    # 筛选名词和形容次，动词
#     processFile(input_path,output_path,"NAV")
    for p in input_path:
        print "Process Folder:",p
        processFile(p,output_path,ssList,"NA")

In [13]:
def main():
    
    # 处理全体数据
#     processAll()
    
    # 处理测试数据
    processTest()    

In [14]:
if __name__ == '__main__':
    main()